In [2]:
# checking what unrolled w computation would be
var("a b c x y", domain='real');
w = vector((a,b,c));
I = diagonal_matrix((x,x,y));
w.cross_product(I*w)

(-b*c*x + b*c*y, a*c*x - a*c*y, 0)

In [3]:
var("gt gr Ks li lj rx rx2 ry ry2 rz rz2 vx vx2 vy vy2 vz vz2", domain='real')
var("qa qb qc qd qa2 qb2 qc2 qd2 ux uy uz Kx Ky",domain="real");
var("wx wy wz wx2 wy2 wz2",domain="real");
ri = vector((rx,ry,rz));
ri2 = vector((rx2,ry2,rz2));
vi = vector((vx,vy,vz));
vi2 = vector((vx2,vy2,vz2));
q = vector((qa,qb,qc,qd));
q2 = vector((qa2,qb2,qc2,qd2));
w = vector((wx,wy,wz));
w2 = vector((wx2,wy2,wz2));
u = vector((ux,uy,uz));
dr = ri2-ri;
dv = vi2-vi;
sq = q2+q
dq = q2-q

In [4]:
# stretch
stretch = .5*li*Ks*(dr.norm()/li - 1)^2
pd_stretch = vector((
    diff(stretch,rx2),
    diff(stretch,ry2),
    diff(stretch,rz2)
));
pd_stretch

(-1.00000000000000*Ks*(rx - rx2)*(sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2)/li - 1)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2), -1.00000000000000*Ks*(ry - ry2)*(sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2)/li - 1)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2), -1.00000000000000*Ks*(rz - rz2)*(sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2)/li - 1)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))

In [5]:
# verify stretch reduction
norm = dr.norm();
pd_stretch_red = -Ks*dr*(1/norm-1/li);
for i in range(3):
    print(bool(pd_stretch_red[i] == pd_stretch[i]))

True
True
True


In [6]:
# bending energy
B = matrix([
    [sq[3],sq[2],-sq[1],-sq[0]],
    [-sq[2],sq[3],sq[0],-sq[1]],
    [sq[1],-sq[0],sq[3],-sq[2]]
]);
udelta = (B*dq)/lj - u
bending = .5*lj*(Kx*(udelta[0]^2 + udelta[1]^2) + Ky*udelta[2]^2)
pd_bending = vector((
    diff(bending,qa2),
    diff(bending,qb2),
    diff(bending,qc2),
    diff(bending,qd2),
));
pd_bending

(-2.00000000000000*(Kx*(qd*(ux + ((qb - qb2)*(qc + qc2) - (qb + qb2)*(qc - qc2) + (qa - qa2)*(qd + qd2) - (qa + qa2)*(qd - qd2))/lj)/lj - qc*(uy - ((qa - qa2)*(qc + qc2) - (qa + qa2)*(qc - qc2) - (qb - qb2)*(qd + qd2) + (qb + qb2)*(qd - qd2))/lj)/lj) + Ky*qb*(uz + ((qa - qa2)*(qb + qb2) - (qa + qa2)*(qb - qb2) + (qc - qc2)*(qd + qd2) - (qc + qc2)*(qd - qd2))/lj)/lj)*lj, -2.00000000000000*(Kx*(qc*(ux + ((qb - qb2)*(qc + qc2) - (qb + qb2)*(qc - qc2) + (qa - qa2)*(qd + qd2) - (qa + qa2)*(qd - qd2))/lj)/lj + qd*(uy - ((qa - qa2)*(qc + qc2) - (qa + qa2)*(qc - qc2) - (qb - qb2)*(qd + qd2) + (qb + qb2)*(qd - qd2))/lj)/lj) - Ky*qa*(uz + ((qa - qa2)*(qb + qb2) - (qa + qa2)*(qb - qb2) + (qc - qc2)*(qd + qd2) - (qc + qc2)*(qd - qd2))/lj)/lj)*lj, 2.00000000000000*(Kx*(qb*(ux + ((qb - qb2)*(qc + qc2) - (qb + qb2)*(qc - qc2) + (qa - qa2)*(qd + qd2) - (qa + qa2)*(qd - qd2))/lj)/lj - qa*(uy - ((qa - qa2)*(qc + qc2) - (qa + qa2)*(qc - qc2) - (qb - qb2)*(qd + qd2) + (qb + qb2)*(qd - qd2))/lj)/lj) - Ky*q

In [7]:
# verify bending reduction
Q2 = matrix([
    [-qd2,qc2,-qb2],
    [-qc2,-qd2,qa2],
    [qb2,-qa2,-qd2],
    [qa2,qb2,qc2]
]);
Q2j = matrix([
    [-qd,qc,-qb],
    [-qc,-qd,qa],
    [qb,-qa,-qd],
    [qa,qb,qc]
]);
udelta_k = vector((
    udelta[0]*Kx,
    udelta[1]*Kx,
    udelta[2]*Ky
));
pd_bend_reduction = -Q2j*2*udelta_k
for i in range(4):
    print(bool(pd_bend_reduction[i].simplify() == pd_bending[i].simplify()))

True
True
True
True


In [8]:
# constraint translation
var("d3x d3y d3z k",domain="real")
d3 = vector((d3x,d3y,d3z))
err = (dr/dr.norm() - d3);
constraint = .5*li*k*err.dot_product(err)
pd_constraint_actual = vector((
    diff(constraint,rx2),
    diff(constraint,ry2),
    diff(constraint,rz2)
));

In [9]:
# verify constraint translation reduction
drr = dr.dot_product(dr);
dr_l2_inv = 1/dr.norm();
dr_norm = dr*dr_l2_inv;
delta = (dr_norm-d3);
pd_constraint = -k*li*dr_l2_inv*(dr*delta.dot_product(dr)/drr - delta)
for i in range(3):
    print(bool(pd_constraint[i] == pd_constraint_actual[i]))

True
True
True


In [10]:
# constraint rotation energy
d3 = vector((2*(qa*qc+qb*qd), 2*(qb*qc-qa*qd), qd*qd+qc*qc-qb*qb-qa*qa))
err = (dr/dr.norm() - d3);
constraint = .5*li*k*err.dot_product(err)
pd_conrot_act = vector((
    diff(constraint,qa),
    diff(constraint,qb),
    diff(constraint,qc),
    diff(constraint,qd)
));
pd_conrot_act

(2.00000000000000*((qa^2 + qb^2 - qc^2 - qd^2 - (rz - rz2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qa + (2*qa*qc + 2*qb*qd + (rx - rx2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qc - (2*qb*qc - 2*qa*qd + (ry - ry2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qd)*k*li, 2.00000000000000*((qa^2 + qb^2 - qc^2 - qd^2 - (rz - rz2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qb + (2*qb*qc - 2*qa*qd + (ry - ry2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qc + (2*qa*qc + 2*qb*qd + (rx - rx2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qd)*k*li, 2.00000000000000*((2*qa*qc + 2*qb*qd + (rx - rx2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qa + (2*qb*qc - 2*qa*qd + (ry - ry2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qb - (qa^2 + qb^2 - qc^2 - qd^2 - (rz - rz2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qc)*k*li, -2.00000000000000*((2*qb*qc - 2*qa*qd + (ry - ry2)/sqrt((rx - rx2)^2 + (ry - ry2)^2 + (rz - rz2)^2))*qa - (2*qa*qc + 2*qb*q

In [27]:
# verify conrot reduction
Q1 = matrix([
    [-qc,qd,qa],
    [-qd,-qc,qb],
    [-qa,-qb,-qc],
    [-qb,qa,-qd]
]);
pd_conrot = 2*k*li*Q1*err;
for i in range(4):
    print(bool(pd_conrot[i] == pd_conrot_act[i]))

True
True
True
True


In [12]:
# translation dissipation friction
vrel = dr*(dv.dot_product(dr))/li^3
disst = .5*li*gt*vrel.dot_product(vrel)
pd_disstran_act = vector((
    diff(disst,vx),
    diff(disst,vy),
    diff(disst,vz),
))
pd_disstran_act

(1.00000000000000*gt*li*(((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(rx - rx2)^3/li^6 + ((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(rx - rx2)*(ry - ry2)^2/li^6 + ((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(rx - rx2)*(rz - rz2)^2/li^6), 1.00000000000000*gt*li*(((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(rx - rx2)^2*(ry - ry2)/li^6 + ((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(ry - ry2)^3/li^6 + ((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(ry - ry2)*(rz - rz2)^2/li^6), 1.00000000000000*gt*li*(((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(rx - rx2)^2*(rz - rz2)/li^6 + ((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(ry - ry2)^2*(rz - rz2)/li^6 + ((rx - rx2)*(vx - vx2) + (ry - ry2)*(vy - vy2) + (rz - rz2)*(vz - vz2))*(rz - rz2)^3/li^6))

In [13]:
# verify disstran reduction
drr = dr.dot_product(dr)
drv = dr.dot_product(dv)
pd_disstran = -gt/li^5 * drr*drv*dr;
for i in range(3):
    print(bool(pd_disstran[i]==pd_disstran_act[i]))

True
True
True


In [14]:
# angular dissipation
# QJ has first column trimmed off
QJ1 = matrix([
    [-qb, -qc, -qd],
    [ qa, -qd,  qc],
    [ qd,  qa, -qb],
    [-qc,  qb,  qa]
]);
QJ2 = matrix([
    [-qb2, -qc2, -qd2],
    [ qa2, -qd2,  qc2],
    [ qd2,  qa2, -qb2],
    [-qc2,  qb2,  qa2]
]);
Bkj1 = matrix([
    [ qd, -qc,  qb, -qa],
    [ qc,  qd, -qa, -qb],
    [-qb,  qa,  qd, -qc]
]);
Bkj2 = matrix([
    [ qd2, -qc2,  qb2, -qa2],
    [ qc2,  qd2, -qa2, -qb2],
    [-qb2,  qa2,  qd2, -qc2]
]);
var("qda qdb qdc qdd qda2 qdb2 qdc2 qdd2"); # q time derivative
qdt = vector((qda,qdb,qdc,qdd));
qdt2 = vector((qda2,qdb2,qdc2,qdd2));

bkjs = Bkj2*qdt2 - Bkj1*qdt;
angdiss = 2*gr/lj*bkjs.dot_product(bkjs);
pd_angdiss_act = vector((
    diff(angdiss,qda2),
    diff(angdiss,qdb2),
    diff(angdiss,qdc2),
    diff(angdiss,qdd2),
));
pd_angdiss_act

(-4*((qb*qda - qb2*qda2 - qa*qdb + qa2*qdb2 - qd*qdc + qd2*qdc2 + qc*qdd - qc2*qdd2)*qb2 + (qc*qda - qc2*qda2 + qd*qdb - qd2*qdb2 - qa*qdc + qa2*qdc2 - qb*qdd + qb2*qdd2)*qc2 + (qd*qda - qd2*qda2 - qc*qdb + qc2*qdb2 + qb*qdc - qb2*qdc2 - qa*qdd + qa2*qdd2)*qd2)*gr/lj, 4*((qb*qda - qb2*qda2 - qa*qdb + qa2*qdb2 - qd*qdc + qd2*qdc2 + qc*qdd - qc2*qdd2)*qa2 + (qd*qda - qd2*qda2 - qc*qdb + qc2*qdb2 + qb*qdc - qb2*qdc2 - qa*qdd + qa2*qdd2)*qc2 - (qc*qda - qc2*qda2 + qd*qdb - qd2*qdb2 - qa*qdc + qa2*qdc2 - qb*qdd + qb2*qdd2)*qd2)*gr/lj, 4*((qc*qda - qc2*qda2 + qd*qdb - qd2*qdb2 - qa*qdc + qa2*qdc2 - qb*qdd + qb2*qdd2)*qa2 - (qd*qda - qd2*qda2 - qc*qdb + qc2*qdb2 + qb*qdc - qb2*qdc2 - qa*qdd + qa2*qdd2)*qb2 + (qb*qda - qb2*qda2 - qa*qdb + qa2*qdb2 - qd*qdc + qd2*qdc2 + qc*qdd - qc2*qdd2)*qd2)*gr/lj, 4*((qd*qda - qd2*qda2 - qc*qdb + qc2*qdb2 + qb*qdc - qb2*qdc2 - qa*qdd + qa2*qdd2)*qa2 + (qc*qda - qc2*qda2 + qd*qdb - qd2*qdb2 - qa*qdc + qa2*qdc2 - qb*qdd + qb2*qdd2)*qb2 - (qb*qda - qb2*qda2 - q

In [15]:
# angdiss reduction verify
pd_angdiss = -4*gr/lj*transpose(-Bkj2)*bkjs
for i in range(4):
    print(bool(pd_angdiss[i] == pd_angdiss_act[i]))

True
True
True
True


In [16]:
# quaternion <-> director analysis
R = matrix([
    [1-2*(qb*qb+qc*qc), 2*(qa*qb-qc*qd), 2*(qa*qc+qb*qd)],
    [2*(qa*qb+qc*qd), 1-2*(qa*qa+qc*qc), 2*(qb*qc-qa*qd)],
    [2*(qa*qc-qb*qd), 2*(qb*qc+qa*qd), 1-2*(qa*qa+qb*qb)]
]);
q2 = q.dot_product(q)-1;
one_mat = diagonal_matrix((q2,q2,q2))
R + one_mat

[ qa^2 - qb^2 - qc^2 + qd^2          2*qa*qb - 2*qc*qd          2*qa*qc + 2*qb*qd]
[         2*qa*qb + 2*qc*qd -qa^2 + qb^2 - qc^2 + qd^2          2*qb*qc - 2*qa*qd]
[         2*qa*qc - 2*qb*qd          2*qb*qc + 2*qa*qd -qa^2 - qb^2 + qc^2 + qd^2]

In [17]:
# other tests....
var("aa bb cc dd")
A = 1-2*(bb+cc)
B = 1-2*(aa+cc)
C = 1-2*(aa+bb)
(-A-B+C+1)/4

cc

In [23]:
Q = matrix([
    [qd, qc, -qb, qa],
    [-qc, qd, qa, qb],
    [qb, -qa, qd, qc],
    [-qa, -qb, -qc, qd]
]);

NameError: name 'simplify_full' is not defined